# Práctica 1 - Clasificación de palabras

In [2]:
import pandas as pd
from matplotlib.colors import ListedColormap
import sklearn.preprocessing
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
#https://scikit-learn.org/stable/modules/model_evaluation.html
import matplotlib.pyplot as plt

import numpy as np
from numpy import mean

from sklearn.svm import SVC

## Preparación de los datos
En esta primera parte de la práctica vamos a preparar los datos para poder hacer la clasificación. Para ello haremos que cada muestra siga el siguiente formato: C1 | C2 | C3 | .. | CN | Y  (siendo C una característica y Y el target).

In [2]:
#Auxiliar function that reformats df to one that is more fitting to Classification Problems
def reformat(dataFrame):
    dataFrame['y']=dataFrame.columns[0]
    dataFrame.rename(columns={dataFrame.columns[0]: 'word'}, inplace=True)
    return dataFrame

#RawData, it has been a little bit formatted before reading the csv, to facilitate the process.
raw=pd.read_csv("data/data.csv")
#Split df
catala, angles= raw.filter(['catala'], axis=1), raw.filter(['angles'], axis=1)
#Reformating
catala=reformat(catala)
angles=reformat(angles)

#Merging
wordsDF=pd.concat([catala,angles], axis=0)
#Shuffle the rows
wordsDF = wordsDF.sample(frac=1).reset_index(drop=True)



### Caracterísiticas
Las características que hemos pensado que pueden ser útiles para la clasificación de las palabras son:
- Cantidad de caracteres (Númerica)
- Proporción de consonantes por vocal (consonantes / vocales) (Numérica)
- Contiene patrones o normas ortográficas de una lengua de las que vamos a clasificar?
    + Doble uso de vocal consecutivamente como es el caso del inglés (Categórica)
    + Acentos en caso de catalán (Categórica)
    + Contiene combinaciones de consonantes (consonant clusters) propias del inglés? (Categórica)

Referencias
- <Consonant_Clusters :https://www.aprendeinglessila.com/2013/09/consonantes-ingles-clusters/>
- <Frecuencia_De_Letras_Usadas_En_Catalan: https://es.sttmedia.com/frecuencias-de-letras-catalan>
- <Frecuencia_De_Letras_Usadas_En_Catalan: https://www3.nd.edu/~busiforc/handouts/cryptography/letterfrequencies.html>

Para añadir las columnas que representen estas características, hemos aplicado las siguientes funciones a las palabras.

In [3]:
#ratio de consonantes y vocales
def ratio (word):
    vocals=0
    for c in word:
        if isVocal(c):
            vocals+=1
    return round(vocals/len(word), 4)

#isVocal?
def isVocal(c):
    if(c=='a' or c=='e' or c=='i' or c=='o' or c=='u'):
        return True
    return 
#gotAccent?
def gotAccent(word):
    #List containing all possible accentuated chars from Catalan
    accentuatedChars=[ord('à'),ord('è'),ord('é'),ord('í'),ord('ò'),ord('ó'),ord('ú')]
    for c in word:
        if ord(c) in accentuatedChars:
            return 1
    return 0
def doubleVocal(word):
    ocurrences=["aa","ee","ii","oo","uu"]
    for oc in ocurrences:
        if word.find(oc)!=-1:
            return 1
    return 0

def enCC(word):
    ocurrences=["sch","spl","shr","squ","thr","spr","scr","sph","th","tw","sw","sk","sm"]
    for oc in ocurrences:
        if word.find(oc)!=-1:
            return 1
    return 0

#Features Adding
wordsDF['ratio']=wordsDF['word'].apply(ratio)
wordsDF['cantidadLetras']=wordsDF['word'].apply(len)
wordsDF['gotAccent']=wordsDF['word'].apply(gotAccent)
wordsDF['doubleVocal']=wordsDF['word'].apply(doubleVocal)
wordsDF['enCC']=wordsDF['word'].apply(enCC)
#Reorganize DF
wordsDF=wordsDF[['word','ratio','cantidadLetras','gotAccent','doubleVocal','enCC','y']]
wordsDF.to_csv('data/definitiveData.csv', index=False)
print(wordsDF.to_string)

<bound method DataFrame.to_string of           word   ratio  cantidadLetras  gotAccent  doubleVocal  enCC       y
0         verb  0.2500               4          0            0     0  catala
1      arribar  0.4286               7          0            0     0  catala
2        capaç  0.4000               5          0            0     0  catala
3           em  0.5000               2          0            0     0  catala
4       màster  0.1667               6          1            0     0  catala
...        ...     ...             ...        ...          ...   ...     ...
1971  filferro  0.3750               8          0            0     0  catala
1972      each  0.5000               4          0            0     0  angles
1973     lluna  0.4000               5          0            0     0  catala
1974      love  0.5000               4          0            0     0  angles
1975    trobar  0.3333               6          0            0     0  catala

[1976 rows x 7 columns]>


### Separación del conjunto de datos en entrenamiento y test
Vamos a escoger dos tercios para el conjunto de entrenamiento y el resto para el test. 

In [21]:
#First we must separate dataframe into X and y format
X=wordsDF.iloc[:,1:6]
X_aux=wordsDF.iloc[:,1:3]
y=wordsDF.iloc[:,6]
#For better perfomance, we scale the data using an standard scaler
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(X_aux)
standardData_aux = scaler.transform(X_aux)
print(standardData_aux)
#Ratio Standarized
print(standardData_aux[:,0])
#NumLletres Standarized
print(standardData_aux[:,1])
standardData=X
standardData['ratio']=standardData_aux[:,0]
standardData['cantidadLetras']=standardData_aux[:,1]
print(standardData)
#Then we separate the data frame in training and test (will be used in chosen model)
X_train, X_test, y_train, y_test = train_test_split(standardData, y, test_size=0.33, random_state=33)

[[-1.07855609 -0.66949106]
 [ 0.37096326  0.94974313]
 [ 0.13884538 -0.12974633]
 ...
 [ 0.13884538 -0.12974633]
 [ 0.95044636 -0.66949106]
 [-0.40249248  0.4099984 ]]
[-1.07855609  0.37096326  0.13884538 ...  0.13884538  0.95044636
 -0.40249248]
[-0.66949106  0.94974313 -0.12974633 ... -0.12974633 -0.66949106
  0.4099984 ]
         ratio  cantidadLetras  gotAccent  doubleVocal  enCC
0    -1.078556       -0.669491          0            0     0
1     0.370963        0.949743          0            0     0
2     0.138845       -0.129746          0            0     0
3     0.950446       -1.748981          0            0     0
4    -1.754620        0.409998          1            0     0
...        ...             ...        ...          ...   ...
1971 -0.064055        1.489488          0            0     0
1972  0.950446       -0.669491          0            0     0
1973  0.138845       -0.129746          0            0     0
1974  0.950446       -0.669491          0            0     0
197

## Modelos
En este problema creemos que el mejor modelo para clasificar las palabras va a ser una SVC con un kernel gaussiano (RBG) ya que tenemos un número de características bastante bajo en comparación al número de muestras. 

Los híperparámetros que deberemos de configurar en el modelo del SVM Lineal son:
- _penalty_
- _loss_
- _C_
- _max_iter_

El resto de híperparámetros no son tan necesarios ya que no se ajustan al problema. Por ejemplo _dual_  no nos va a dar ninguna diferencia, ya que es un booleano que indica que algoritmo usar. Se usa **dual = False** cuando el número de muestras es mayor al de características, que es nuestro caso. <br>Tampoco usaremos *multi_class* ya que en este experimento solo usaremos las clases "catalan" y "angles".

Para el ajuste de estos híperparámetros haremos un _nested cross-validation_ que consiste en combinar **GridSearch** junto a **K-Folding**, pero en este caso usaremos la variación _Stratified_ para mantener una distribución balanceada de las clases en cada fold. 

In [22]:
#Modelo
SVM_rbf = SVC(C=50,kernel='rbf',max_iter=1000, gamma=1)

### K-Folding
Lo primero que debemos de hacer es decidir qué valor de _k_ vamos a utilizar. Normalmente se utilizan valores entre 5 y 10. Vamos a probar entre estos rangos y ver qué valor nos da mejores resultados.

In [23]:
def evaluate_model(cv):
    #https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation
    score = cross_val_score(SVM_rbf,X,y,scoring='accuracy',cv=cv,n_jobs=-1)
    return mean(score),score.min(),score.max()


#Range to be tested
folds = range (5,11)
for k in folds:
    #Shuffle not needed because dataset is not sorted by class 
    cv=StratifiedKFold(n_splits=k, shuffle=False, random_state=1)
    k_mean, k_min, k_max = evaluate_model(cv)
    print(f'-> folds={k}, accuracy = {round(k_mean,4)}, ({round(k_min,4)}, {round(k_max,4)})')



-> folds=5, accuracy = 0.6462, (0.6025, 0.6793)
-> folds=6, accuracy = 0.6386, (0.5957, 0.6818)
-> folds=7, accuracy = 0.6452, (0.6064, 0.689)
-> folds=8, accuracy = 0.6392, (0.587, 0.6802)
-> folds=9, accuracy = 0.6427, (0.589, 0.6849)
-> folds=10, accuracy = 0.6301, (0.5635, 0.7056)


# GridSearchLayout

In [ ]:


#GridSearchCV(SVM_rbf, [1,0], cv=None, verbose=0)


# configure the cross-validation procedure
#cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
cv_outer = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(standardData):
 # split data
 X_train, X_test = standardData[train_ix, :], standardData[test_ix, :]
 y_train, y_test = y[train_ix], y[test_ix]
 # configure the cross-validation procedure
 cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
 # define the model
 SVM_rbf = SVC(C=50,kernel='rbf',max_iter=1000, gamma=1)
 # define search space
 param_grid = dict()
 param_grid['n_estimators'] = [10, 100, 500]
 param_grid['max_features'] = [2, 4, 6]
 # define search
 search = GridSearchCV(SVM_rbf, param_grid, scoring='accuracy', cv=cv_inner, refit=True)
 #search = GridSearchCV(SVM_rbf, param_grid, cv=None, verbose=0)
 # execute search
 result = search.fit(X_train, y_train)
 # get the best performing model fit on the whole training set
 best_model = result.best_estimator_
 # evaluate model on the hold out dataset
 yhat = best_model.predict(X_test)
 # evaluate the model
 acc = accuracy_score(y_test, yhat)
 # store the result
 outer_results.append(acc)
 # report progress
 print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))